# 1dconv cnnでモデルつくる 

In [1]:
import os
import sys
import random
from collections import defaultdict
from typing import Any, Callable, Dict, List, Optional, Tuple, Type, Union

import numpy as np
import pandas as pd
import polars as pl
import matplotlib.pyplot as plt
import torch
import pytorch_lightning as ptl
from pytorch_lightning.loggers import CSVLogger
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from torch.utils.data import DataLoader, Dataset


sys.path.append("../src")

from utils import timer, seed_everything
from utils.io import save_pickle, load_pickle
from metric import optimize_f1_score

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [2]:
class PSPDataset(Dataset):

    def __init__(self, data: pd.DataFrame, is_test: bool = False) -> None:
        self.elapsed_time_diff = torch.tensor(data["elapsed_time_diff"].tolist(), dtype=torch.float32)
        self.event_name = torch.tensor(data["event_name"].tolist(), dtype=torch.long)
        self.level = torch.tensor(data["level"].tolist(), dtype=torch.long)
        self.fqid = torch.tensor(data["fqid"].tolist(), dtype=torch.long)
        self.room_fqid = torch.tensor(data["room_fqid"].tolist(), dtype=torch.long)

        if is_test:
            self.labels = torch.empty(len(data), 2)
        else:
            self.labels = torch.tensor(data["correct"].tolist(), dtype=torch.float32)

    def __len__(self) -> int:
        return len(self.labels)

    def __getitem__(self, idx: int) -> Tuple[torch.Tensor, Dict[str, torch.Tensor]]:
        return (
            self.labels[idx], 
            {
                "elapsed_time_diff": self.elapsed_time_diff[idx],
                "event_name": self.event_name[idx],
                "level": self.level[idx],
                "fqid": self.fqid[idx],
                "room_fqid": self.room_fqid[idx],
            }
        )

def get_dataloader(
    data: pd.DataFrame, 
    batch_size: int = 128, 
    shuffle: bool = True, 
    drop_last: bool = False,
    is_test: bool = False
) -> DataLoader:
    dataset = PSPDataset(data, is_test)
    return DataLoader(
        dataset,  # type: ignore
        batch_size=batch_size, 
        shuffle=shuffle, 
        drop_last=drop_last,
    )

In [3]:
class ConvEncoder(torch.nn.Module):
    def __init__(self, input_dim: int, conv_dim: int = 64, num_feature: int = 128) -> None:
        super(ConvEncoder, self).__init__()

        self.conv = torch.nn.Sequential(
            torch.nn.Conv1d(input_dim, conv_dim, kernel_size=5),
            torch.nn.BatchNorm1d(conv_dim),
            torch.nn.LeakyReLU(inplace=True),
        )
        self.fc = torch.nn.Linear(conv_dim * 2, num_feature)
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x  = self.conv(x)
        std = torch.std(x, dim = 2)
        mean = torch.mean(x, dim = 2)
        x = torch.cat([std, mean], dim = 1)
        x = self.fc(x)
        return x


class PSPLightningModule(ptl.LightningModule):
    
    def __init__(
        self, 
        continuous_dim: int = 1, 
        event_name_embedding_dim: int = 8,
        level_embedding_dim: int = 8,
        fqid_embedding_dim: int = 8,
        room_fqid_embedding_dim: int = 8,
        conv_num_feature: int = 128,
        output_dim: int = 3,
        learning_rate: float = 1e-2,
    ) -> None:
        super(PSPLightningModule, self).__init__()
        self.save_hyperparameters()
        self.learning_rate = learning_rate

        self.event_name_embedding = torch.nn.Embedding(100, event_name_embedding_dim, padding_idx=0)
        self.level_embedding = torch.nn.Embedding(100, level_embedding_dim, padding_idx=0)
        self.fqid_embedding = torch.nn.Embedding(100, fqid_embedding_dim, padding_idx=0)
        self.room_fqid_embedding = torch.nn.Embedding(100, room_fqid_embedding_dim, padding_idx=0)

        conv_input_dim = (continuous_dim + event_name_embedding_dim + level_embedding_dim + fqid_embedding_dim + room_fqid_embedding_dim)
        self.conv_encoder = ConvEncoder(input_dim=conv_input_dim, conv_dim=128, num_feature=conv_num_feature)
        self.fc = torch.nn.Sequential(
            torch.nn.Linear(conv_num_feature, conv_num_feature // 2),
            torch.nn.ReLU(inplace=True),
            torch.nn.Linear(conv_num_feature // 2, output_dim),
        )

        self.criteria = torch.nn.BCEWithLogitsLoss()
        self.validation_step_outputs: Dict[str, List[torch.Tensor]] = defaultdict(list)
        self.predict_step_outputs: List[torch.Tensor] = []
    
    def forward(self, feat: Dict[str, torch.Tensor]) -> torch.Tensor:
        continuous = feat["elapsed_time_diff"].unsqueeze(1)
        event_name = self.event_name_embedding(feat["event_name"])
        level = self.level_embedding(feat["level"])
        fqid = self.fqid_embedding(feat["fqid"])
        room_fqid = self.room_fqid_embedding(feat["room_fqid"])

        x_cat = torch.cat([event_name, level, fqid, room_fqid], dim=2).transpose(1, 2)
        x = torch.cat([continuous, x_cat], dim=1)
        x = self.conv_encoder(x)
        x = self.fc(x)
        return x

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate, weight_decay=1e-4)
        return optimizer
    
    def training_step(self, batch: Any, batch_idx: int) -> torch.Tensor:
        label, feat = batch
        pred = self(feat)
        loss = self.criteria(pred, label)
        self.log("train_loss", loss, on_step=True, prog_bar=True, logger=True)
        return loss
    
    def validation_step(self, batch: Any, batch_idx: int) -> torch.Tensor:
        label, feat = batch
        pred = self(feat)
        loss = self.criteria(pred, label)
        self.validation_step_outputs["pred"].append(pred)
        self.validation_step_outputs["label"].append(label)
        return loss
    
    def on_validation_epoch_end(self) -> None:
        pred = torch.cat(self.validation_step_outputs["pred"], dim=0)
        label = torch.cat(self.validation_step_outputs["label"], dim=0)
        loss = self.criteria(pred, label)
        self.log("valid_loss", loss, on_step=False, prog_bar=True, logger=True)
    
    def predict_step(self, batch: Any, batch_idx: int) -> Any:
        _, feat = batch
        pred = self(feat)
        self.predict_step_outputs.append(pred)
        return pred

In [4]:
def pad_sequence(seq: List, max_seq_len: int, pad_value: Any) -> List:
    if len(seq) >= max_seq_len:
        return seq[-max_seq_len:]
    else:
        return [pad_value] * (max_seq_len - len(seq)) + seq


def create_features(X: pd.DataFrame, y: pd.DataFrame, max_seq_len: int) -> pd.DataFrame:
    features = X.groupby("session_id")[["elapsed_time_diff", "event_name", "level", "fqid", "room_fqid"]].agg(
        lambda x: pad_sequence(list(x), max_seq_len, 0)
    )
    labels = y.groupby("session")["correct"].apply(list)
    data = pd.merge(labels, features, how="left", left_index=True, right_index=True) 
    return data

In [5]:
train = pl.read_parquet("../data/preprocessing/train.parquet")
uniques_map = load_pickle("../data/preprocessing/uniques_map.pkl")

y_train = pl.read_parquet("../data/feature/y_train_fold_0.parquet")
y_valid = pl.read_parquet("../data/feature/y_valid_fold_0.parquet")

In [6]:
max_seq_len = 512
batch_size = 512

output_dims = {"0-4": 3, "5-12": 10, "13-22": 5}

pred_all_level_groups = []
label_all_level_groups = []
for level_group in ["0-4", "5-12", "13-22"]:
    _train = train.filter(train["level_group"] == level_group).to_pandas()
    _y_train = y_train.filter(y_train["level_group"] == level_group).to_pandas()
    _y_valid = y_valid.filter(y_valid["level_group"] == level_group).to_pandas()


    _train = _train.assign(
        elapsed_time_diff=_train["elapsed_time"].fillna(0).clip(0.0, 3.6e6),
    )

    _uniques_map = uniques_map[level_group]
    cols_to_encode = ["event_name", "level", "fqid", "room_fqid"]
    for col in cols_to_encode:
        _train[col] = _train[col].fillna(f"{col}_null").map({
            k: int(i + 1) for i, k in enumerate(_uniques_map[col] + [f"{col}_null"])
        })
    
    train_data = create_features(_train, _y_train, max_seq_len)
    valid_data = create_features(_train, _y_valid, max_seq_len)

    train_dataloader = get_dataloader(train_data, batch_size=batch_size, shuffle=True, drop_last=True)
    vlaid_dataloader = get_dataloader(valid_data, batch_size=batch_size, shuffle=False, drop_last=False)

    model = PSPLightningModule(output_dim=output_dims[level_group])
    logger = CSVLogger(save_dir="logs", name="psp")
    checkpoint_callback = ModelCheckpoint("./ckpt", monitor="valid_loss", save_top_k=1, mode="min")
    checkpoint_ealry_stopping = EarlyStopping(monitor="valid_loss", patience=3, mode="min")
    trainer = ptl.Trainer(
        max_epochs=500, 
        accelerator="gpu",
        check_val_every_n_epoch=1,
        log_every_n_steps=10,
        logger=logger,
        callbacks=[checkpoint_callback, checkpoint_ealry_stopping],
    )
    trainer.fit(model, train_dataloader, vlaid_dataloader)

    _pred = trainer.predict(model, vlaid_dataloader)
    pred = torch.cat(_pred, dim=0).sigmoid().numpy()

    label = vlaid_dataloader.dataset.labels.numpy()

    pred_all_level_groups.append(pred)
    label_all_level_groups.append(label)

/opt/conda/lib/python3.10/site-packages/torch/cuda/__init__.py:173: UserWarning: 
NVIDIA GeForce RTX 3060 with CUDA capability sm_86 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_37 sm_60 sm_70 sm_75 compute_70 compute_75.
If you want to use the NVIDIA GeForce RTX 3060 GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  warnings.warn(incompatible_device_warn.format(device_name, capability, " ".join(arch_list), device_name))
/opt/conda/lib/python3.10/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:612: UserWarning: Checkpoint directory /workspace/notebook/ckpt exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
/opt/conda/lib/python3.10/site-packages/lightning_fabric/loggers/csv_logs.py:188: UserWarning: Experiment logs directory logs/psp/version_0 exists and is not empty. Previous log files in this directory will

Sanity Checking: 0it [00:00, ?it/s]

/opt/conda/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/opt/conda/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

/opt/conda/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, predict_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Predicting: 0it [00:00, ?it/s]

In [8]:
pred_all = np.concatenate(pred_all_level_groups, axis=1)
label_all = np.concatenate(label_all_level_groups, axis=1)

for level in range(label_all.shape[1]):
    score, threshold = optimize_f1_score(label_all[:, level], pred_all[:, level])

    print(f"level-{level}: F1score={score:.4f} threshold={threshold:.4f}")

score, threshold = optimize_f1_score(label_all.ravel(), pred_all.ravel())
print(f"overall: F1score={score:.4f} threshold={threshold:.4f}")

level-0: F1score=0.5696 threshold=0.7374
level-1: F1score=0.5246 threshold=0.9596
level-2: F1score=0.5432 threshold=0.8889
level-3: F1score=0.6199 threshold=0.7273
level-4: F1score=0.5992 threshold=0.5455
level-5: F1score=0.6097 threshold=0.7071
level-6: F1score=0.5983 threshold=0.6869
level-7: F1score=0.5581 threshold=0.5859
level-8: F1score=0.6029 threshold=0.6364
level-9: F1score=0.5877 threshold=0.5354
level-10: F1score=0.5889 threshold=0.6162
level-11: F1score=0.5718 threshold=0.7980
level-12: F1score=0.5752 threshold=0.3434
level-13: F1score=0.5795 threshold=0.6869
level-14: F1score=0.5904 threshold=0.4646
level-15: F1score=0.5400 threshold=0.7172
level-16: F1score=0.5587 threshold=0.6667
level-17: F1score=0.5557 threshold=0.8990
overall: F1score=0.6710 threshold=0.6364
